In [143]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib notebook

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics




from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB

In [144]:
df = pd.read_csv('791531.csv', encoding='latin-1')
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


#### 1. Only got 13000 location data

In [145]:
df.shape

(20050, 26)

In [146]:
df.isna().sum()

_unit_id                     0
_golden                      0
_unit_state                  0
_trusted_judgments           0
_last_judgment_at           50
gender                      97
gender:confidence           26
profile_yn                   0
profile_yn:confidence        0
created                      0
description               3744
fav_number                   0
gender_gold              20000
link_color                   0
name                         0
profile_yn_gold          20000
profileimage                 0
retweet_count                0
sidebar_color                0
text                         0
tweet_coord              19891
tweet_count                  0
tweet_created                0
tweet_id                     0
tweet_location            7484
user_timezone             7798
dtype: int64

In [147]:
df['gender'].value_counts()

female     6700
male       6194
brand      5942
unknown    1117
Name: gender, dtype: int64

In [148]:
df.shape

(20050, 26)

In [149]:
df.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'gender', 'gender:confidence', 'profile_yn',
       'profile_yn:confidence', 'created', 'description', 'fav_number',
       'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'profileimage',
       'retweet_count', 'sidebar_color', 'text', 'tweet_coord', 'tweet_count',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [150]:
df['gender:confidence'].describe()

count    20024.000000
mean         0.882756
std          0.191403
min          0.000000
25%          0.677800
50%          1.000000
75%          1.000000
max          1.000000
Name: gender:confidence, dtype: float64

In [151]:
sum(df['gender:confidence'] == 1)

13926

In [152]:
#drop unknown gender, gender confidence < 1, and null loaction

df.drop(df[df['gender'] == 'unknown'].index, inplace = True)
df.drop(df[df['gender:confidence'] != 1].index, inplace = True)
df['tweet_location'].dropna(inplace = True)

In [153]:
df['gender'].value_counts()

female    5367
male      4653
brand     3784
Name: gender, dtype: int64

In [157]:
df['tweet_location'].isna().sum()

0

In [158]:
#pick features for modeling
df1 = df[['text', 'gender']]

In [159]:
df1

,text,gender
0,Robbie E Responds To Critics After Win Against...,male
1,ÛÏIt felt like they were my friends and I was...,male
3,Hi @JordanSpieth - Looking at the url - do you...,male
4,Watching Neighbours on Sky+ catching up with t...,female
5,"Ive seen people on the train with lamps, chair...",female
...,...,...
20044,Need A Ride Home From Practice _Ù÷Ô_Ù÷Ô_Ù÷ÔAnd...,female
20045,"@lookupondeath ...Fine, and I'll drink tea too...",female
20046,Greg Hardy you a good player and all but don't...,male
20047,You can miss people and still never want to se...,male


In [160]:
#further clean gender as binary
drop_items_idx = df1[(df1['gender'] == 'unknown') | (df1['gender'] == 'brand')].index
df1.drop (index = drop_items_idx, inplace = True)

df1['gender'].value_counts()

/Users/Lee_Lu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


female    5367
male      4653
Name: gender, dtype: int64

In [165]:
df1['gender'] = df1['gender'].map({ 'male':1, 'female':0})

/Users/Lee_Lu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [166]:
df1['gender'].value_counts()

0.0    5367
1.0    4653
Name: gender, dtype: int64

In [168]:
df1.dropna(inplace = True)

/Users/Lee_Lu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [169]:
df1.isna().sum()

text      0
gender    0
dtype: int64

In [170]:
X_train, X_test, y_train, y_test = train_test_split(df1['text'], 
                                                    df1['gender'], 
                                                    test_size=0.20,
                                                    random_state=100)

In [171]:
X_train.shape

(8016,)

In [172]:
#Countvector---------------
pipe_cv_rfc = Pipeline([('vect', CountVectorizer(stop_words = 'english')),
                     ('clf', RandomForestClassifier(random_state = 100))])

pipe_cv_lr = Pipeline([('vect', CountVectorizer(stop_words = 'english')),
                     ('clf', LogisticRegression(random_state = 100))])

pipe_cv_svm = Pipeline([('vect', CountVectorizer(stop_words = 'english')),
                         ('clf', SVC(random_state = 100))])


pipe_cv_mnb = Pipeline([('vect', CountVectorizer(stop_words = 'english')),
                        ('clf', MultinomialNB())])

#Tfidf---------------
pipe_tv_rfc = Pipeline([('vect', TfidfVectorizer(stop_words = 'english')),
                     ('clf', RandomForestClassifier(random_state = 100))])

pipe_tv_lr = Pipeline([('vect', TfidfVectorizer(stop_words = 'english')),
                       ('clf', LogisticRegression(random_state = 100))])
                       
pipe_tv_svm = Pipeline([('vect', TfidfVectorizer(stop_words = 'english')),
                        ('clf', SVC(random_state = 100))])


pipe_tv_mnb = Pipeline([('vect', TfidfVectorizer(stop_words = 'english')),
                        ('clf', MultinomialNB())])

In [173]:
param_range = [i for i in np.arange(1,10,1)]
param_range_fl = [1, 0.5, 0.25, 0.1]
 

grid_params_rfc = [{'clf__criterion': ['gini', 'entropy'],
                    'vect__min_df': param_range,
                    'vect__max_df':[11, 20],
                    'clf__n_estimators':[100]}]

grid_params_lr = [{'clf__penalty': ['l1', 'l2'],
                   'clf__C': param_range_fl,
                   'clf__solver': ['liblinear']}]

grid_params_svm = [{'clf__kernel': ['linear', 'rbf'], 
                    'clf__C': param_range}]

grid_params_mnb =  [{'clf__alpha': param_range}]

In [175]:
# Construct GridSearches

gs_cv_rfc = GridSearchCV(estimator = pipe_cv_rfc, 
                         param_grid = grid_params_rfc,
                         scoring ='accuracy',
                         cv = 5,
                         n_jobs = -1) 

gs_cv_lr = GridSearchCV(estimator = pipe_cv_lr,
                        param_grid = grid_params_lr,
                        scoring='accuracy',
                        cv = 5,
                        n_jobs = -1)

gs_cv_svm = GridSearchCV(estimator = pipe_cv_svm,
                          param_grid = grid_params_svm,
                          scoring='accuracy',
                          cv = 5,
                          n_jobs = -1)

gs_cv_mnb = GridSearchCV(estimator = pipe_cv_mnb, 
                         param_grid = grid_params_mnb,
                         scoring ='accuracy',
                         cv = 5,
                         n_jobs = -1) 
#Tfidf----------------
gs_tv_rfc = GridSearchCV(estimator = pipe_tv_rfc, 
                         param_grid = grid_params_rfc,
                         scoring ='accuracy',
                         cv = 5,
                         n_jobs = -1) 

gs_tv_lr = GridSearchCV(estimator = pipe_tv_lr, 
                         param_grid = grid_params_lr,
                         scoring ='accuracy',
                         cv = 5,
                         n_jobs = -1) 

gs_tv_svm = GridSearchCV(estimator = pipe_tv_svm, 
                         param_grid = grid_params_svm,
                         scoring ='accuracy',
                         cv = 5,
                         n_jobs = -1) 

gs_tv_mnb = GridSearchCV(estimator = pipe_tv_mnb, 
                         param_grid = grid_params_mnb,
                         scoring ='accuracy',
                         cv = 5,
                         n_jobs = -1) 

In [ ]:
# Label and compose the models for training
grids = [gs_cv_rfc, gs_cv_lr, gs_cv_svm, gs_cv_mnb, gs_tv_rfc, gs_tv_lr, gs_tv_svm, gs_tv_mnb]

#grid_dict = {0: 'CountVect + RandomForest', 
             1: 'CountVect + LogisticRegression',
             2: 'CountVect + SVC',
            3: 'CountVect + MultinomialNB',
             4: 'Tfidf + RandomForest',
             5: 'Tfidf + LogisticRegression',
             6: 'Tfidf + SVC',
             7: 'Tfidf + MultinomialNB'}

In [195]:
#Label and compose the models for training
grids = [gs_cv_mnb, gs_tv_mnb]

grid_dict = {0: 'CountVect + MultinomialNB',
             1: 'Tfidf + MultinomialNB',
            }

In [196]:
def grid_fiting(X_train, X_test, y_train, y_test, grids, grid_dict):
    best_acc = 0
    best_id = 0
    best_gs = ''
    for names, gs in enumerate(grids):
        gs.fit(X_train, y_train)
        y_pred = gs.predict(X_test)
        print('----------------------------------------------')
        print('%s' % names, '%s' % grid_dict[names])
        print('Best training accuracy score: %s' % gs.best_score_)
        print('Test set accuracy score with best params: %s' % accuracy_score(y_test, y_pred))
        if accuracy_score(y_test, y_pred) > best_acc:
            best_acc = accuracy_score(y_test, y_pred)
            best_gs = gs
            best_id = names
            #storage report
            best_report = metrics.classification_report(y_test, y_pred)
            
    print('              ')
    print('==================SUMMARY=====================')
    print('Best CLF: %s' % best_id, grid_dict[best_id])
    print('Best Test Accuracy: %s' % best_acc)
    print('Best Params: %s' % best_gs.best_params_)
    print('              ')
    print('Classification Report:')
    print('              ')
    print(best_report)
    

In [197]:
grid_fiting(X_train, X_test, y_train, y_test, grids, grid_dict)

----------------------------------------------
0 CountVect + MultinomialNB
Best training accuracy score: 0.6190119760479041
Test set accuracy score with best params: 0.6127744510978044
----------------------------------------------
1 Tfidf + MultinomialNB
Best training accuracy score: 0.6014221556886228
Test set accuracy score with best params: 0.594311377245509
              
==================SUMMARY=====================
Best CLF: 0 CountVect + MultinomialNB
Best Test Accuracy: 0.6127744510978044
Best Params: {'clf__alpha': 1}
              
Classification Report:
              
              precision    recall  f1-score   support

         0.0       0.60      0.77      0.68      1044
         1.0       0.64      0.44      0.52       960

    accuracy                           0.61      2004
   macro avg       0.62      0.61      0.60      2004
weighted avg       0.62      0.61      0.60      2004

